In [ ]:
import os

import tensorflow as tf
from tensorflow import keras
import argparse
from imutils import paths
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob
matplotlib.use("Agg")

from tensorflow.keras import backend as K
from tensorflow.keras import utils as np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import  ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import seaborn as sns

In [4]:
Hg = 150
Lng = 150

INIT_LR = 0
BS = 16
EPOCHS = 20

BASE_DIR = '.\\Vegetable Images'

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen_flow = train_datagen.flow_from_directory(BASE_DIR + '\\train', target_size=(Hg, Lng), batch_size=BS, seed=42, class_mode='categorical')

Found 15001 images belonging to 15 classes.


In [14]:
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen_flow = valid_datagen.flow_from_directory(BASE_DIR + '\\validation', target_size=(Hg, Lng), batch_size=BS, seed=42, class_mode='categorical')

Found 3000 images belonging to 15 classes.


In [28]:
model = Sequential()
model.add(Conv2D(15, (11, 11), input_shape=(Lng, Hg, 3),
        padding='same', kernel_regularizer=l2(INIT_LR)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2
model.add(Conv2D(36, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Layer 5
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 6
model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Layer 7
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Layer 8
model.add(Dense(15))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

In [33]:
model.compile(loss='categorical_crossentropy', metrics=['acc', 'AUC'], optimizer='adam')
H = model.fit(train_datagen_flow, validation_data=valid_datagen_flow, steps_per_epoch=len(train_datagen_flow), 
          validation_steps=len(valid_datagen_flow), epochs=EPOCHS, verbose=2)

Epoch 1/20
938/938 - 32s - loss: 0.0717 - acc: 0.9838 - auc: 0.9467 - val_loss: 0.1698 - val_acc: 0.9497 - val_auc: 0.9387 - 32s/epoch - 34ms/step
Epoch 2/20
938/938 - 29s - loss: 0.0748 - acc: 0.9831 - auc: 0.9427 - val_loss: 0.2406 - val_acc: 0.9363 - val_auc: 0.9062 - 29s/epoch - 31ms/step
Epoch 3/20
938/938 - 29s - loss: 0.0608 - acc: 0.9869 - auc: 0.9455 - val_loss: 0.0390 - val_acc: 0.9903 - val_auc: 0.9774 - 29s/epoch - 31ms/step
Epoch 4/20
938/938 - 30s - loss: 0.0587 - acc: 0.9861 - auc: 0.9441 - val_loss: 0.0918 - val_acc: 0.9750 - val_auc: 0.9353 - 30s/epoch - 32ms/step
Epoch 5/20
938/938 - 30s - loss: 0.0655 - acc: 0.9849 - auc: 0.9405 - val_loss: 0.0481 - val_acc: 0.9857 - val_auc: 0.9542 - 30s/epoch - 32ms/step
Epoch 6/20
938/938 - 30s - loss: 0.0473 - acc: 0.9898 - auc: 0.9436 - val_loss: 0.4333 - val_acc: 0.8587 - val_auc: 0.8803 - 30s/epoch - 32ms/step
Epoch 7/20
938/938 - 30s - loss: 0.0601 - acc: 0.9845 - auc: 0.9387 - val_loss: 0.4009 - val_acc: 0.8797 - val_auc: 0.

In [36]:
N = np.arange(0, EPOCHS)

plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_accuracy")
plt.plot(N, H.history["val_acc"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('result.png')